In [ ]:
import time
import csv

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def setDate(date, date2, new):
    driver.find_element(By.ID, "collapse-step-1").click()
    driver.implicitly_wait(1)
    if (new):
        driver.find_element(By.CSS_SELECTOR, "#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp1 > div.tab-btn-inner.tab1 > a").click()
        driver.implicitly_wait(1)
    driver.find_element(By.CSS_SELECTOR, "#srch-tab1 > div > div.radio-btn-wp.col6 > span:nth-child(1) > label").click()
    driver.implicitly_wait(1)
    input = driver.find_element(By.CSS_SELECTOR, "#search-begin-date")
    driver.implicitly_wait(1)
    for i in range(10):
        input.send_keys(Keys.BACK_SPACE)
    input.send_keys(date)
    input.send_keys(Keys.TAB)
    input = driver.find_element(By.CSS_SELECTOR, "#search-end-date")
    driver.implicitly_wait(1)
    for i in range(10):
        input.send_keys(Keys.BACK_SPACE)
    input.send_keys(date2)
    input.send_keys(Keys.ENTER)
    driver.find_element(By.CSS_SELECTOR, '#search-foot-div > div.foot-btn > button.btn.btn-search.news-search-btn.news-report-search-btn').click()
    driver.implicitly_wait(10)
    time.sleep(1)


def getNextDate(y, m, d):
    zero = [4, 6, 9, 11]
    one = [1, 3, 5, 7, 8, 10, 12]
    y_res = y
    m_res = m
    d_res = d
    if ((m in zero) and d == 30):
        m_res += 1
        d_res = 1
    elif ((m in one) and d == 31):
        if (m == 12):
            y_res += 1
            m_res = 1
        else:
            m_res += 1
        d_res = 1
    elif (m == 2 and d == 28):
        if (((y % 4 == 0) and (y % 100 != 0)) or (y % 400 == 0)):
            d_res += 1
        else:
            m_res += 1
            d_res = 1
    elif (m == 2 and d == 29):
        m_res += 1
        d_res = 1
    else:
        d_res += 1

    return y_res, m_res, d_res

def dateToString(y, m, d):
    y_str = str(y)
    m_str = str(m)
    d_str = str(d)
    if (m / 10 < 1):
        m_str = str(0) + str(m)
    if (d / 10 < 1):
        d_str = str(0) + str(d)
    return y_str + m_str + d_str

def getNextMonth(y, m, d):
    y_res, m_res, d_res = getNextDate(y, m, d)
    for i in range(30):
        y_res, m_res, d_res = getNextDate(y_res, m_res, d_res)
    return y_res, m_res, d_res

driver_file = 'C:/Users/leejihui/chromedriver'
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome(driver_file)
driver.implicitly_wait(10)
url = 'https://www.bigkinds.or.kr/v2/news/search.do'
driver.get(url)
driver.find_element(By.ID, "collapse-step-1").click()
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, '#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp2 > div.tab-btn-inner.tab4 > a').click()
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, 'li[data-id="1"] label').click()
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, 'div[class="foot-btn"] > button:last-child').click()
driver.implicitly_wait(10)
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, '#select2').click()
#driver.implicitly_wait(10)
#driver.execute_script("arguments[0].click()", select)
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, '#select2 > option:nth-child(4)').click()
driver.implicitly_wait(10)
#driver.execute_script("arguments[0].click()", submit)
#driver.implicitly_wait(10)

year = 2018
month = 1
day = 1

year2, month2, day2 = getNextMonth(year, month, day)

y_dest = 2022
m_dest = 12
d_dest = 2

setDate(dateToString(year, month, day), dateToString(year2, month2, day2), True)

f = open('data.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)

while (dateToString(y_dest, m_dest, d_dest) >= dateToString(year, month, day)):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.implicitly_wait(5)
    page = soup.select('#news-results-tab > div.data-result-ac.pc-only.paging-v2-wrp.top > div.data-result-btm > div > div > div > div > div:nth-child(6) > div')[0]['data-page']
    pageNum = int(page)
    #print(pageNum)
    j = 0
    while (j < pageNum):
        newsId = []
        newsTitle = []
        newsDate = []
        newsContent = []

        ids = soup.select('.news-detail p')
        for i in ids:
            newsId.append(i['data-newsid'])
            print(i['data-newsid'])
            print(i.text.strip())
        titles = soup.select('.title-elipsis')
        for i in titles:
            newsTitle.append(i.text.strip())
            print(i.text.strip())
        dates = soup.select('.info p:nth-of-type(1)')
        for i in dates:
            newsDate.append(i.text.strip())
            print(i.text.strip())
        for i in range(len(newsId)):
            title = driver.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(' + str(i+1) + ') > div > div.cont > a')
            driver.execute_script("arguments[0].click()", title)
            driver.implicitly_wait(1)
            #time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents = soup.select('#news-detail-modal > div > div > div.modal-body > div > div.news-view-body')
            driver.implicitly_wait(1)
            temp = []
            for k in contents:
                temp.append(k.text.strip())
                print(k.text.strip())
            newsContent.append(temp)
            driver.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > button').click()
            driver.implicitly_wait(1)

        for k in range(len(newsId)):
            temp = [newsId[k], newsTitle[k], newsDate[k], newsContent[k]]
            wr.writerow(temp)

        next = driver.find_element(By.CSS_SELECTOR, '#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(7) > a')
        driver.execute_script("arguments[0].click()", next)
        driver.implicitly_wait(10)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        driver.implicitly_wait(5)
        j += 1

    year, month, day = getNextDate(year2, month2, day2)
    year2, month2, day2 = getNextMonth(year, month, day)
    setDate(dateToString(year, month, day), dateToString(year2, month2, day2), False)

<ipython-input-2-f6a7bfdc89fc>:84: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_file)


01100201.20180201102944001
조카를 흉기로 찔러 잔혹하게 살해한 외삼촌이 항소심에서도 무기징역을 선고받았다. 조카가 “집에서 나가 달라”고 요구하자 앙심을 품고 살인을 저질렀다.  대구고법 형사1부(부장판사 박준용)는 40대 조카의 목과 얼굴을 흉기로 50여차례 찔러 살해한 혐의로 구속기소된 A(63)씨 항소심에서 1심과 같이 무기징역을 선고했다고 1일 밝혔다.  미국 시민..
01400201.20180201095330001
지난해 6월 대전 지역 사립대 대학생 A씨는 메신저 앱을 통해 중학생 B양과 만나 합의 하에 성관계를 맺었다.하지만 A씨는 이후 B양을 상대로 성관계 장면이 촬영된 블랙박스 영상을 유포하겠다며 15만 원을 요구했다. B양은 응하지 않았고 이 사실이 경찰에 알려지면서 A씨는 덜미가 잡혔다. 중학생과 성관계를 맺고 협박해 돈을 뺏으려 한 혐의(공갈미수)로..
01500601.20180201093004001
서지현 검사의 검찰 내 성폭력 사건 폭로를 계기로 검찰발 미투(Me Too) 운동이 확산되고 있는 가운데 시민사회단체들이 1일 전국 16개 검찰청 앞에서 진상 규명을 촉구하는 기자회견을 열었다. 이날 오전 대구지검 앞에서 대구경북여성단체연합, 민주사회를위한변호사모임 대구지부 등 50개 지역시민사회단체 회원들이 성폭력 고발 운동인 미투 캠페인의 상징인 하..
01500601.20180201092959004
*해시태그: #우디앨런 #불륜 #사랑과전쟁 #막장드라마 *명대사: ‘이건 현실이 아니야, 나는 웨이트리스 역을 연기하고 있는 거야.’ *줄거리: 뉴욕, 코니아일랜드에서 웨이트리스로 일하는 지니(케이트 윈슬렛)는 해변의 안전요원 미키(저스틴 팀버레이크)와 밀애를 나누는 사이다. 한때 연극배우 이력을 가지고 있는 지니는 남편이 있지만 언젠가 미키와 ..
01500601.20180201092955001
전국 여성단체들이 잇따라 검찰 내 성추행 사건을 폭로한 서지현 검사에 대한 지지를 표하고 나섰다. 성범죄 사실을 고백한 피해자

누나 집에 얹혀 살다 조카 살해… 비정한 외삼촌 ‘무기징역’
스마트폰 악용한 성매매…아이들이 위험하다
전국 검찰청 앞 진상규명 'Me Too' 캠페인
[이사강의 LIKE A MOVIE] 노장 감독 우디 앨런식 우아한 막장 드라마 ‘원더 휠’
서지현 검사 폭로에 전국 여성단체 '미투'(Me Too) 지지
검찰, 대구은행 채용비리 의혹 수사
박근혜 진박감별 불법 여론조사 관여 혐의로 추가 기소, 기소 혐의 모두 21개
전직 여경도, 道의원도 “나도 당했다”… 미투운동 확산
‘부산 여중생 폭행’ 가해 10대들 소년부 송치 ‘형벌 대신 보호처분’
김영란법 원재료 기준 탓 설 대목에도 침울한 농가
검찰발 '미투' 계기… 공공부문 직장 성폭력 특별점검한다
‘친박 공천개입’ 또 기소, 박근혜 혐의 21개
일본 기업, 한국 반도체 기술 훔쳐 공장 차려
최교일 “성추행 은폐 주장 명예훼손”
“성폭력 사건엔 늘 ‘꽃뱀 프레임’… 저열한 물타기죠”
[Hi #이슈]기안84, 여성혐오 논란…'나혼자' 하차 요구 VS 과민 반응
[천종호의 판사의 길] 희생양(犧牲羊)과 ‘마이너스1’의 제의(祭儀)
‘여혐 논란’ 휩싸인 기안84… ‘나 혼자 산다’ 하차 여론까지
“마트 직원 불친절” 술 취한 60대 남성 분신 시도
'미스티', 3회까지 시청등급 19세 "김남주-지진희 격정멜로 위해"
선박 견적 부풀려 부당 대출받고 고래까지 잡은 간 큰 일당들
몰카로 여경 협박한 ‘악질 동료’ 실형
김성태 “문재인 정부, 무능 말고 보여준 게 없다”
최교일, 우병우, 원세훈, 이영학의 공통점은? ‘선비의 고장’ 영주가 괴롭다
서지현 검사 측 “법무부, 내부 성폭력 안이하게 생각한 것”
수사 정보주고 뇌물받은 경찰관 징역형
내연녀 5살 아들 폭행·실명시킨 20대 중형
“졸지 말고 집중해”
‘친부모에 의한 비극’ 막을 사회 안전망 절실
송도국제화복합단지개발 전 사업본부장 등 3명 불구속 기소
전두환 신군부, 삼청교육대생 무인도 수용 검토했었다
與 개헌초안에 촛불혁명 명시…토지공개념 조항 강화하기로
“법무장관

서지현 검사가 근무하는 창워지검 통영지청에 서지현 검사를 응원하는 꽃바구니가 놓여졌다. 연합뉴스


						

					전국 여성단체들이 잇따라 검찰 내 성추행 사건을 폭로한 서지현 검사에 대한 지지를 표하고 나섰다. 성범죄 사실을 고백한 피해자에게 응원을 보내는 '미투'(Me Too, 나도 당했다)가 사회적으로 확대되고 있다는 분석이다. 한국여성단체연합, 한국여성의전화, 한국성폭력상담소 등 전국 50여개 여성인권단체는 1일 서울 서초구 대검찰청과 도봉구 북부지검 등 전국 15개 지역에서 동시다발 기자회견을 가졌다. 이들은 서지현 검사의 성폭력 피해 고발에 응원과 지지를 보내면서 범죄 사실에 대한 철저한 진상규명과 가해자 처벌을 요구했다. 경남여성단체연합 등 30여개 경남 여성단체 소속 회원 50여명도 같은 날 오전 창원시 성산구 사파동 서지현 검사가 근무하는 통영지청의 상위 관청인 창원지검 청사 앞에서 기자회견을 열고 "용기를 내 성추행 사건을 외부에 알린 창원지검 통영지청 서지현 검사를 격려하고 지지를 보낸다"고 밝혔다. 이어 "민간전문가가 참여하는 특별조사위원회를 구성, 진상을 철저히 밝히고 서 검사가 2차 피해를 입지 않도록 조치하라"고 검찰에 요구했다.이날 오전 제주여민회를 주축으로 한 6개 제주지역 여성단체 회원 20여명도 제주지방검찰청 앞에서 손 피켓을 들고 “검찰은 조직 내 성폭력 사건의 진상을 철저히 규명하고 대책을 마련하라"고 목소리를 높였다. 이들은 민간전문가를 포함한 특별조사위원회 구성은 물론 현재 설립이 추진되고 있는 고위공직자범죄수사처(공수처)를 통해서도 공정한 수사가 이뤄져야 한다고 주장했다. 남성들도 가세했다. 성평등 문화 확산에 앞장서는 남성들의 모임인 '성평등 보이스'도 같은 날 입장문을 발표하고 성희롱·성폭력 근절을 촉구했다. 이들은 입장문에서 "성희롱·성폭력을 개인의 일탈로 취급하기보다는 오랜 기간 지속된 구조적 문제로 봐야 한다"며 "최근 피해 여성들의 폭로 고발 움직임은 사회 구조적 문제 해결을 위한 과정의 시작이다.

120여회 불법 여론조사 실시친박계 인사 당선 배후 지원김기춘ㆍ조윤선도 추가 기소화이트리스트 수사 사실상 마무리朴 1심 선고는 빨라야 3월 말박근혜 전 대통령이 지난 20대 총선에 불법 관여한 혐의로 추가 기소됐다. 검찰은 김기춘 전 청와대 비서실장, 조윤선 전 청와대 정무수석 등 박근혜 정부 인사들을 대거 재판에 넘기며, 이른바 ‘화이트리스트’ 및 국가정보원 뇌물 상납 사건 관련 조사를 사실상 마무리했다. 서울중앙지검 특수3부(부장 양석조)는 1일 박 전 대통령을 공직선거법 위반(부정선거운동) 혐의로 추가 기소했다. 박 전 대통령은 2016년 4ㆍ13총선을 앞두고 당시 새누리당(자유한국당 전신) 지지도가 높은 지역에 ‘진박’(진실한 친박계) 인사들을 대거 당선시키기 위해 공천과정에 개입한 혐의를 받고 있다. 박 전 대통령은 앞서 기소된 삼성 뇌물수수, 미르ㆍK스포츠재단 대기업 출연 강요, 국정원 특수활동비 수수 등 20개 혐의에 더해 총 21개 혐의로 재판을 받게 됐다.검찰 조사 결과 청와대는 비박계 인사들을 공천에서 배제할 목적으로 친박계 의원들과 협의해 ‘진박 리스트’를 만들고 이들의 당선 가능성을 점검하는 불법 여론조사를 120회 가까이 실시한 것으로 드러났다. 청와대는 당 공천 규칙과 관련한 대응 지침 등 선거기획 문건도 만들어 당시 새누리당 공천관리위원회 측에 전달하거나, 진박 인사를 지원하기 위해 경쟁 후보자에게 출마 포기를 종용하는 등 경선에 개입한 것으로 파악됐다. 박 전 대통령이 여론조사 결과를 수 차례 보고 받고 관련 지시를 내리는 등 관여했다는 것이 검찰 판단이다. 검찰은 또 새누리당 총선 후보 경선과 관련해 불법 여론조사 비용 중 5억원을 국정원으로부터 받은 혐의(특정범죄가중처벌법상 뇌물 등)로 현기환 전 청와대 정무수석과 후임인 김재원 자유한국당 의원을 불구속 기소했다. 이들은 청와대 예산으로 총 12억여원에 달하는 여론조사 비용을 충당하기 어렵자 국정원에 자금 지원을 지시한 혐의를 받고 있다. 검찰은 국정원으로부터 뇌물을 받은 박근혜

...

= 검찰 내 성추행 사건 진상규명 및 피해회복 조사단장으로 임명된 조희진 서울 동부지검장이 1일 서울 송파구 동부지검에서 열린 기자간담회에 들어서고 있다. 서지현 창원지검 통영지청 검사는 지난 26일 검찰 내부망(이프로스)에 글을 올려 지난 2010년 10월 안태근 당시 법무부 정책기획단장에게 성추행을 당한 뒤 인사상 불이익을 받았다는 내용을 폭로했다. 이에 문무일 검찰총장은 1월31일 '성추행 사건 진상규명 및 피해회복 조사단'을 발족하고 조희진 서울동부지검장(56·사법연수원 19기)을 조사단장으로 임명했다. 2018.2.1/뉴스1뉴스1 제공
= 검찰 내 성추행 사건 진상규명 및 피해회복 조사단장으로 임명된 조희진 서울 동부지검장이 1일 서울 송파구 동부지검에서 열린 기자간담회에 들어서고 있다. 서지현 창원지검 통영지청 검사는 지난 26일 검찰 내부망(이프로스)에 글을 올려 지난 2010년 10월 안태근 당시 법무부 정책기획단장에게 성추행을 당한 뒤 인사상 불이익을 받았다는 내용을 폭로했다. 이에 문무일 검찰총장은 1월31일 '성추행 사건 진상규명 및 피해회복 조사단'을 발족하고 조희진 서울동부지검장(56·사법연수원 19기)을 조사단장으로 임명했다. 2018.2.1/뉴스1뉴스1 제공
중간선거를 앞두고 미국 민주당 내에서 힐러리 클린턴의 '몸값'이 오르고 있지만 최근 미 전역의 이른바 '미투(Me too)' 캠페인이 클린턴의 발목을 잡을 것이란 관측이 제기됐다. 민주당 측 관계자는 지난달 31일(현지시간) CNN과 인터뷰에서 "올해 클린턴이 확실히 (선거의) 길에 나설 것"이라며 "이미 많은 (민주당) 후보들로부터 '콜'을 받고 있다"고 말했다. 클린턴에게 러브콜이 쏟아지는 이유는 민주당 유권자 사이에서 클린턴의 인기가 여전한 데다 클린턴의 행보에 언론의 관심이 집중되는 만큼 선거 유세에서 흥행을 불러일으킬 수 있기 때문이다. 클린턴도 선거에 적극 개입하는 모양새다. 익명을 요구한 한 측근은 지난해 클린턴이 톰 페레즈 민주당 전국위원장을 최소 두 차례 이상 만

= 탈세·횡령 의혹을 받고 있는 이중근 부영그룹 회장이 1일 오전 서울 서초구 중앙지검에 피의자 신분으로 출석하고 있다. 지난 31일 이 회장은 조사를 받기 위해 검찰에 출석했지만 건강상의 이유로 11시간 만에 귀가해 이날 재소환됐다. 2018.2.1/뉴스1뉴스1 제공
이명박 정부 당시 청와대가 국가인권위원회(인권위) 직원의 블랙리스트를 만들었다는 의혹을 인권위가 즉각 조사해야 한다는 권고가 나왔다.인권위 외부 자문기구인 국가인권위원회혁신위원회(혁신위)는 이 같은 내용을 포함한 최종 권고안을 인권위에 제시했다고 1일 밝혔다. 과거 반성과 재발 방지를 위한 방안, 인권위 내 비정규직 문제 해결 등 9건 등이다. 혁신위는 "청와대의 인권위 블랙리스트 전달은 인권위의 독립성을 훼손한 것"이라며 "인권위 조사관들의 독립적 활동을 보장하는 방안을 마련해야 한다"고 밝혔다.혁신위는 2008년부터 2015년까지 인권위의 독립성이 훼손됐던 사례를 조사해 발표했다. 혁신위가 조사한 사건은 △2010년 청와대가 전달한 인권위 직원의 블랙리스트 사건 △인권위 직원 부당징계, 성소수자 배움터 대관 사건 △용산 철거민 사망사건 법원의견제출 지연 사건 △유엔 자유권 쟁점목록 의견서 축소 제출 사건 △PD수첩 명예훼손 관련 의견 제출 사건 등이다.혁신위는 인권위 직원 부당징계 사건과 관련해 구성원의 내부 비판을 보장하고 노조 활동을 인정하라고 권고했다. 성소수자 혐오 행사 대관 사건에 대해서는 성적지향과 성별 정체성을 이유로 한 혐오 선동과 폭력, 사회적 낙인과 차별을 배제한다는 공식 입장을 밝히라고 했다.혁신위는 "권력의 눈치를 보며 인권침해에 면죄부를 주는 등 설립 목적에 반하는 활동을 제대로 평가하고 다시는 이런 일이 발생하지 않도록 과거 사건을 조사했다"고 밝혔다.인권위 내 비정규직 고용 문제와 차별을 해소하라는 권고도 나왔다. 혁신위에 따르면 인권위에는 무기계약직, 기간제 등 비정규직이 임금 등에 차별을 받고 있다. 혁신위는 인권위가 비정규직을 정규직화하겠다고 선언하고 이를 위

검찰내 성추행 피해 사실을 폭로한 서지현 창원지검 통영지청 검사(사법연수원 33기)에 대해 법조계 안팎에서 지지와 응원이 쏟아지고 있다. 용기있는 '미투'(Me too)에 대해 '당신과 함께 한다'는 뜻의 '위드유'(with You)로 화답하는 셈이다.서 검사의 사법연수원 동기(33기) 225명은 1일 성명을 통해 "다른 피해자는 없는지도 샅샅이 조사해야 할 것이며 어렵게 용기 내어 피해를 밝힌 서 검사에게 어떠한 불이익도 없을 것을 요구한다"며 검찰 내부의 성폭력 피해에 대해 철저한 진상 규명을 촉구했다.이들은 "그의 바람대로 동일한 피해가 발생하지 않도록 검찰 내 성차별적인 조직 문화를 근본적으로 성찰하고 오히려 성폭력 피해자가 불이익 받는 불공정한 관행과 절차를 뜯어고칠 것을 요구한다"며 "지난 8년간 그가 감당해야했을 고통과 절망을 짐작조차 하기 어렵다"고 했다. 또 "그동안 함께 하지 못한 미안함을 담아 지금부터라도 용기 내어 준 그의 곁에 함께 하겠다"며 "이 사건을 계기로 아직도 말 꺼내지 못한 다른 모든 성폭력 피해자들이 상처를 치유할 수 있는 힘과 용기를 얻기 바란다"고 덧붙였다.서 검사의 동문인 이화여대 법조인 및 이대 법대·법학전문대학원 동창회 294명 역시 지난달 31일 성명을 내고 "성폭력 피해자가 대중 앞에 나서는 것은 참으로 힘겨운 일"이라며 "우리는 용기 있는 모습을 보여준 서지현 검사를 응원한다"고 밝혔다. 또 "왜 검사의 신분을 가진 피해자가 피해 발생 직후 제 목소리를 낼 수 없었는지, 피해자의 목소리가 조직 내에서 묻힐 수밖에 없었던 이유는 무엇인지 밝혀내야 할 것"이라며 서 검사에 대한 인사상 불이익이 있었는지에 대해 엄중히 조사할 것을 촉구했다. 아울러 "이 사건의 본질을 훼손하는 수근거림으로 피해자에게 돌이킬 수 없는 2차 피해가 발생하지 않도록 검찰조직은 그 책임을 다해야 할 것"이라고 강조했다.정유미 서울중앙지검 공판3부장도 최근 검찰내부망(이프로스)에 글을 올려 후배 검사들을 독려한 것으로 알려졌다. 정 부장검사는

눈 주변 뼈가 부러져 5세 아이를 2시간 동안 방치한 어학원 교사에게 집행유예가 선고됐다. 수원지법 형사 11단독 김도요 판사는 아동복지법위반(아동학대) 등의 혐의로 기소된 교사 김모(35·여)씨에게 징역 4개월에 집행유예 2년을 선고하고, 사회봉사 40시간과 아동학대 치료강의 40시간 수강을 명령했다고 1일 밝혔다. 용인의 한 어학원 교사인 김씨는 지난해 2월 7일 오후 2시20분께 자신이 근무하는 어학원 강당에서 A(5)군이 다른 아동과 부딪힌 뒤 바닥에 엎드려 울고 있는데도 발로 등을 2차례 툭툭 찬 뒤 병원에 데려가지 않고 방치한 혐의로 기소됐다. A군은 혼자 교실로 돌아와 2시간 동안 책상에 엎드려 우는 등 고통을 호소하다가 귀가해 병원에서 정치 4주의 안와바닥 골정상 진단을 받았다. 재판부는 “피고인의 방치로 5세에 불과한 피해자는 상당한 정신적·육체적 고통을 겪었을 것으로 보이고, 후유증이 발생할 가능성도 있다”라며 “그럼에도 피고인은 자신의 사정만을 내세우며 피해자와 부모에게 충분히 사과하지도 않았다”라고 판결했다. 이어 “다만, 피고인이 피해자의 상해에 직접적인 원인을 제공한 것은 아니며 여러 아동을 동시에 훈육하는 고도의 주의를 요하는 업무 중 미필적 고의로 범행에 이른 것으로 보인다”면서 “피고인이 피해자의 상해에 직접적인 원인을 제공한 것은 아닌 점 등을 참작했다”고 양형 이유를 설명했다. 한편, 재판부는 보호·감독 책임을 소홀히 해 함께 기소된 원장 김모(56·여)씨에게는 벌금 1천만 원을 선고했다. 변근아기자/gaga99@joongboo.com
이천시에도 수도권 광역버스 운행이 가능해졌다. 시는 지난 31일 국토교통부가 2018년 업무보고를 통해 “수도권 광역버스를 이천시까지 운행할 수 있도록 제도를 개선하겠다”고 1일 발표했다. 현행 ‘여객자동차 운송사업법 시행규칙’에 따르면 시내버스운송사업은 특별시·광역시·시 또는 군의 단일 행정구역을 운행하는 사업으로 하되, 지역주민의 편의 또는 지역 여건상 특히 필요하다고 인정되는 경우에는 해당

ㆍ폭로 검사에 업무능력 ‘뒷담화’ ㆍSNS 고발 여성 ‘역고소’ 시달려 ㆍ법적 보호 시스템 마련 시급해“성추행 사건 문제제기 때문이 아니라 일을 못해서 원치 않는 곳으로 발령받은 것이다.” “당시에는 가만히 있다가 인사에 불만을 품고 뒤늦게 나선 것이다.” “원래 평이 안 좋았다.”검찰 고위간부에게 당한 성추행 피해를 폭로한 서지현 검사에 대해 검찰 내 일각에서 이 같은 내용의 ‘피해자 낙인찍기’가 벌어지고 있다. 서울지역의 한 검사는 “사석에서 서 검사에 대한 안 좋은 ‘풍문’을 들은 적이 있다”며 “피해자가 내부에서 문제해결이 안되자 8년 만에 어렵게 용기를 냈는데 사건과 상관없는 ‘뜬소문’이 더 부각되는 것 같아 안타깝다”고 말했다.서 검사 측은 1일 ‘피해자 업무능력에 대한 부정적 소문 확산, 검찰의 적극적 조치를 요청합니다’라는 제목의 입장문을 내고 “근거 없는 소문의 확산은 피해자에 대한 2차 가해 행위”라고 밝혔다. 서 검사 측은 입장문에서 법무부 장관 표창 2회(2009년, 2012년), 우수사례 선정 11회, 미담사례 선정 1회 등 서 검사의 포상 경력을 공개하며 서 검사의 업무능력이 떨어진다는 주장에 반박했다. 지난달 29일 서 검사의 폭로로 성폭력 피해 사실을 공개적으로 알리는 ‘미투(Me Too) 운동’에 대한 사회적 관심이 높아졌지만, 어렵게 ‘미투’를 외친 피해자는 ‘이번에도’ 2차 가해에 노출됐다. 2차 가해는 범죄 피해자에게 평소 행실이 피해를 자초했다고 말하거나 피해 사실 자체를 인정하지 않는 등 모욕을 주는 것을 말한다. 대다수의 성폭력 피해 고발자들은 근거 없이 품성이나 능력을 폄하하는 조직 구성원들의 ‘뒷담화’부터 명예훼손·무고 등의 혐의로 가해자가 제기하는 ‘역고소’까지 다양한 형태의 2차 가해에 시달린다. ‘미투’ 이후 고발자들을 보호하는 사회적 시스템의 마련이 시급하다는 지적이 나오는 이유다.한국에서 미투 운동은 2016년 사회관계망서비스(SNS)에서 ‘#○○계_내_성폭력’이라는 해시태그로 조직 내 성폭력 피해를 공론화